In [2]:
from keras.saving import register_keras_serializable
@register_keras_serializable()
def r2_keras(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res / (SS_tot + K.epsilon()))

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import MeanSquaredError

def build_model(layers=[64, 32], input_dim=len(X_train)):
    model = Sequential()
    model.add(Dense(layers[0], input_dim=input_dim, activation='relu'))
    for layer in layers[1:]:
        model.add(Dense(layer, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError(), r2_keras])
    return model

NameError: name 'X_train' is not defined

In [5]:
import joblib
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import StackingRegressor

#model_gradient = joblib.load('./gradient_bossting.pkl')
#model_neural = tf.keras.models.load_model('./neural_network.keras', custom_objects={'r2_keras': r2_keras})
with open('stacking_model-temp.pkl', 'rb') as f:
    loaded_model: StackingRegressor = joblib.load(f)



AttributeError: Can't get attribute 'build_model' on <module '__main__'>